# Etapa 5 – Apoio à Avaliação da IA

<p align="center">
    <img src="https://img.icons8.com/ios-filled/100/artificial-intelligence.png" width="80" alt="Ícone IA"/>
</p>
Subetapa: Filtragem dos Parágrafos para Geração de Perguntas

<img src="https://img.icons8.com/ios-filled/50/000000/right--v1.png" width="24" style="vertical-align:middle; margin-right:8px;"/> Objetivo da Subetapa
Durante a preparação do corpus para a geração automática de perguntas, observou-se que alguns arquivos .txt, mesmo após as etapas anteriores de limpeza, ainda continham trechos irrelevantes. Esses trechos, quando enviados ao modelo de NLP, causavam lentidão no processamento e geravam perguntas mal formuladas, com baixa utilidade para fins de validação.

Para resolver esse gargalo, foi desenvolvido um script de filtragem semântica automatizada. Ele identifica e remove parágrafos com conteúdo não técnico ou administrativo, como nomes de autores, contatos institucionais, fichas catalográficas, endereços e notas editoriais.

**O que o Script Faz**
O script executa as seguintes tarefas:

- Lê o arquivo `paragrafos_validos_corpus.xlsx`, que contém todos os parágrafos extraídos dos arquivos .txt considerados válidos estruturalmente.
- Aplica uma função de filtragem semântica (`paragrafo_relevante`):
    - Remove parágrafos curtos (com menos de 6 palavras).
    - Exclui parágrafos que contenham padrões de texto como:
        - Nomes de autores e responsáveis técnicos;
        - Endereços e e-mails;
        - Informações de publicação (tiragem, edição, versão);
        - Trechos genéricos como “anexo”, “índice”, “apêndice”;
        - Informações institucionais repetitivas (ex: “Ministério da Saúde”).

**Cria dois novos arquivos:** <img src="https://img.icons8.com/ios-filled/50/000000/document--v1.png" width="24" style="vertical-align:middle; margin-left:8px;"/>

*paragrafos_filtrados_corpus.xlsx: apenas com os parágrafos considerados relevantes.

relatorio_filtragem_paragrafos.xlsx: com estatísticas da filtragem (totais e percentuais) e amostras dos parágrafos excluídos e mantidos.*

# Diagnóstico Visual de Duplicados e Nomes Corrompidos

Este script realiza uma varredura completa nos arquivos `.txt` localizados na pasta `corpus_final` (ou outra indicada pelo usuário), com o objetivo de identificar:

- **Arquivos duplicados por conteúdo**: a partir do cálculo de hash parcial dos arquivos (1 MB inicial), o script detecta textos que possuem exatamente o mesmo conteúdo, ainda que estejam com nomes diferentes.
- **Arquivos com nomes suspeitos ou corrompidos**: são detectadas repetições anômalas em nomes como `arquivo_txt_txt_txt_2023.txt`, sugerindo um nome mais limpo.

### Funcionalidades principais

- **Interface gráfica interativa**: o usuário escolhe a pasta desejada via botão e visualiza os resultados diretamente na tela.
- **Log automático**: os resultados do diagnóstico são salvos em um arquivo `log_diagnostico.csv`, contendo os arquivos suspeitos de duplicação ou com nomes inconsistentes.
- **Execução segura**: o script apenas **lê** os arquivos — nenhuma alteração é feita nos nomes ou conteúdos durante o diagnóstico.

### Principais funções do script

- `calcular_hash()`: gera uma hash para os primeiros 1MB do conteúdo do arquivo, permitindo identificar duplicações de forma eficiente.
- `normalizar_nome()`: detecta e propõe correções para nomes de arquivos com repetições incorretas de padrões.
- `diagnosticar_pasta()`: realiza a varredura dos arquivos e imprime os resultados na interface, além de registrar um log.
- `iniciar_interface()`: cria a interface gráfica com campos de entrada, botões e área de visualização dos resultados.

Essa etapa é essencial para garantir que a base de dados textual final não contenha arquivos redundantes nem nomes mal formatados, facilitando a organização e os próximos passos do projeto.


In [3]:
#carrega as bibliotecas necessárias
%pip install transformers sentencepiece tqdm

import pandas as pd # Importa a biblioteca pandas para manipulação de dados
import re # Importa a biblioteca re para expressões regulares
import os # Importa a biblioteca os para interações com o sistema operacional
import hashlib # Importa a biblioteca hashlib para calcular hashes de arquivos
import csv # Importa a biblioteca csv para manipulação de arquivos CSV
from pathlib import Path # Importa a classe Path da biblioteca pathlib para manipulação de caminhos de arquivos
import tkinter as tk # Importa a biblioteca tkinter para criar interfaces gráficas
from tkinter import filedialog, messagebox, scrolledtext # Importa componentes específicos da biblioteca tkinter para diálogos de arquivos, mensagens e caixas de texto roláveis
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE # Importa a expressão regular para caracteres ilegais em nomes de arquivos do openpyxl
import torch # Importa a biblioteca torch para computação em tensores
from transformers import T5Tokenizer, T5ForConditionalGeneration # Importa o tokenizador e o modelo T5 da biblioteca transformers
from datetime import datetime# Importa a classe datetime da biblioteca datetime para manipulação de datas e horas
from tqdm import tqdm # Importa a biblioteca tqdm para exibir barras de progresso

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Extração e limpeza de texto Diagnóstico visual de duplicados e nomes corrompidos
# === FUNÇÕES AUXILIARES ===

def calcular_hash(caminho_arquivo, limite=1024*1024):#esta etapa tem como iobjetivo calcular o hash de um arquivo para identificar duplicados
    """Calcula hash de uma parte do arquivo (1 MB por padrão) para diagnóstico rápido."""
    sha256 = hashlib.sha256()
    with open(caminho_arquivo, 'rb') as f:
        sha256.update(f.read(limite))
    return sha256.hexdigest()

def normalizar_nome(nome):#essa etapa busca corrigir nomes de arquivos que podem estar corrompidos ou com formatação estranha
    """Remove repetições consecutivas de sufixos."""
    padrao = r'(txt_\w+\d{4})+'
    return re.sub(padrao, lambda m: m.group(1), nome)

def diagnosticar_pasta(pasta_raiz, saida_texto):
    pasta_raiz = Path(pasta_raiz)
    hash_map = {}
    log_linhas = []
    log_path = pasta_raiz / "log_diagnostico.csv"

    saida_texto.insert(tk.END, f"🔍 Diagnóstico em: {pasta_raiz}\n")
    saida_texto.insert(tk.END, "Apenas leitura — nenhum arquivo será alterado.\n\n")
    saida_texto.update()

    for caminho_arquivo in pasta_raiz.rglob("*.txt"):
        hash_arquivo = calcular_hash(caminho_arquivo)
        nome_original = caminho_arquivo.name
        caminho_pasta = caminho_arquivo.parent

        # Verifica duplicidade de conteúdo
        if hash_arquivo in hash_map:
            saida_texto.insert(tk.END, f"⚠️ Possível duplicado: {nome_original} (igual a {hash_map[hash_arquivo].name})\n")
            log_linhas.append(["DUPLICADO", str(caminho_arquivo), "Mesma hash de", str(hash_map[hash_arquivo])])
        else:
            hash_map[hash_arquivo] = caminho_arquivo

        # Verifica nome corrompido
        nome_corrigido = normalizar_nome(nome_original)
        if nome_corrigido != nome_original:
            saida_texto.insert(tk.END, f"🔁 Nome suspeito: {nome_original} → Sugerido: {nome_corrigido}\n")
            log_linhas.append(["NOME_CORROMPIDO", str(caminho_arquivo), "Sugerido", nome_corrigido])

    with open(log_path, mode='w', newline='', encoding='utf-8') as log_csv:
        writer = csv.writer(log_csv)
        writer.writerow(["Tipo", "Arquivo", "Info", "Referência/Sugestão"])
        writer.writerows(log_linhas)

    saida_texto.insert(tk.END, f"\n📄 Log salvo em: {log_path}\n")
    saida_texto.insert(tk.END, "✅ Diagnóstico concluído.\n")
    saida_texto.update()

# === INTERFACE GRÁFICA ===

def iniciar_interface():
    def escolher_pasta():
        pasta = filedialog.askdirectory(title="Selecione a pasta 'corpus_final'")
        if pasta:
            entrada_pasta.delete(0, tk.END)
            entrada_pasta.insert(0, pasta)

    def executar_diagnostico():
        pasta_escolhida = entrada_pasta.get()
        if not os.path.isdir(pasta_escolhida):
            messagebox.showerror("Erro", "Selecione uma pasta válida.")
            return
        saida_texto.delete(1.0, tk.END)
        diagnosticar_pasta(pasta_escolhida, saida_texto)

    janela = tk.Tk()
    janela.title("🔎 Diagnóstico de Arquivos Duplicados e Nomes Corrompidos")

    tk.Label(janela, text="📁 Pasta Base:").pack(pady=5)
    entrada_pasta = tk.Entry(janela, width=60)
    entrada_pasta.pack()
    tk.Button(janela, text="Selecionar Pasta", command=escolher_pasta).pack(pady=5)

    tk.Button(janela, text="🔍 Executar Diagnóstico", command=executar_diagnostico, bg="blue", fg="white").pack(pady=10)

    saida_texto = scrolledtext.ScrolledText(janela, width=100, height=25)
    saida_texto.pack(padx=10, pady=10)

    janela.mainloop()

# Rodar
if __name__ == "__main__":
    iniciar_interface()


#  Ações de Correção com Base no Diagnóstico

<p align="center">
    <img src="https://img.icons8.com/fluency/96/checked--v1.png" width="80" alt="Correção"/>
</p>

Este script executa as ações sugeridas pelo diagnóstico anterior (`log_diagnostico.csv`), removendo arquivos duplicados reais e renomeando aqueles com nomes corrompidos. Ele deve ser rodado **logo após a Etapa 4**.

### O que o script faz

- **Lê o arquivo de log** gerado na etapa anterior (`log_diagnostico.csv`), que contém os arquivos identificados como duplicados ou com nomes suspeitos.
- **Remove arquivos duplicados** de fato, com base no conteúdo (hash), mantendo apenas uma cópia.
- **Renomeia arquivos** com nomes corrompidos, normalizando sufixos repetidos. Se o nome sugerido já existir, o script adiciona um sufixo incremental (`_1`, `_2`, etc.) para evitar sobrescrita.
- **Gera um novo log (`log_execucao.csv`)**, registrando todas as ações realizadas ou ignoradas (com justificativa).

### Detalhes técnicos

- Os caminhos dos arquivos são tratados com `Path` para garantir compatibilidade e segurança.
- Em caso de erro (como falha ao excluir ou renomear), o erro é capturado e registrado no log de execução.
- O script imprime no terminal as ações realizadas para acompanhamento visual.

### Exemplo de ações realizadas

| Tipo           | Descrição                                               |
|----------------|----------------------------------------------------------|
| REMOVIDO       | Arquivo duplicado foi deletado                          |
| RENOMEADO      | Arquivo com nome corrompido foi renomeado corretamente  |
| IGNORADO       | Arquivo não encontrado no caminho indicado              |
| ERRO           | Algum erro impediu a ação (detalhes no log)             |

Esta etapa complementa o diagnóstico anterior ao aplicar efetivamente as correções necessárias na pasta `corpus_final`, preparando os arquivos para uso limpo e padronizado no projeto IAVS.


In [ ]:
#executar_log_diagnostico remove duplicatas reais e renomeia arquivos com nome corrompido.Logo após o diagnóstico

# === CONFIGURAÇÃO ===
PASTA_BASE = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")  # ajuste se necessário
CAMINHO_LOG_DIAGNOSTICO = PASTA_BASE / "log_diagnostico.csv"# caminho do log de diagnóstico
CAMINHO_LOG_EXECUCAO = PASTA_BASE / "log_execucao.csv" # caminho do log de execução

# === LEITURA DO LOG ===
with open(CAMINHO_LOG_DIAGNOSTICO, encoding='utf-8') as f: # abre o arquivo de log de diagnóstico
    leitor = csv.DictReader(f)
    acoes = list(leitor)

# === EXECUÇÃO DAS AÇÕES ===
log_execucao = [] # lista para armazenar o log de execução

for linha in acoes: # itera sobre cada linha do log de diagnóstico
    tipo = linha['Tipo']
    caminho_str = linha['Arquivo']
    caminho_arquivo = Path(caminho_str)

    if not caminho_arquivo.exists(): # verifica se o arquivo existe
        log_execucao.append(["IGNORADO", caminho_str, "Arquivo não encontrado"])
        continue

    if tipo == "DUPLICADO": # se o tipo for duplicado
        try:
            caminho_arquivo.unlink()
            log_execucao.append(["REMOVIDO", caminho_str, "Duplicata removida"])
            print(f"🗑️ Removido: {caminho_str}")
        except Exception as e: # tenta remover o arquivo
            log_execucao.append(["ERRO", caminho_str, f"Erro ao remover: {e}"])

    elif tipo == "NOME_CORROMPIDO": # se o tipo for nome corrompido
        sugestao = linha['Referência/Sugestão']
        novo_caminho = caminho_arquivo.parent / sugestao
        contador = 1
        while novo_caminho.exists(): # verifica se o novo caminho já existe
            novo_caminho = caminho_arquivo.parent / f"{sugestao}_{contador}.txt"
            contador += 1
        try:# tenta renomear o arquivo
            caminho_arquivo.rename(novo_caminho)
            log_execucao.append(["RENOMEADO", caminho_str, f"Novo nome: {novo_caminho.name}"])
            print(f"✏️ Renomeado: {caminho_arquivo.name} → {novo_caminho.name}")
        except Exception as e:
            log_execucao.append(["ERRO", caminho_str, f"Erro ao renomear: {e}"])

# === SALVA LOG FINAL ===
with open(CAMINHO_LOG_EXECUCAO, mode='w', newline='', encoding='utf-8') as f: # abre o arquivo de log de execução
    writer = csv.writer(f)
    writer.writerow(["Ação", "Arquivo", "Detalhes"])
    writer.writerows(log_execucao)

print(f"\n✅ Execução concluída. Log salvo em: {CAMINHO_LOG_EXECUCAO}")


✅ Execução concluída. Log salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\log_execucao.csv


# Extração e Validação de Parágrafos Válidos

<p align="center">
    <img src="https://img.icons8.com/fluency/96/checked-checkbox.png" width="80" alt="Validação de Parágrafos"/>
</p>

Este script percorre todos os arquivos `.txt` na pasta `corpus_final`, extrai os parágrafos brutos e filtra apenas os **parágrafos considerados válidos**, com o objetivo de construir uma base estruturada para análise e geração de perguntas.

### Objetivos da etapa

- Quebrar corretamente os textos em parágrafos sem ruídos de quebra de linha mal formatada.
- Eliminar blocos irrelevantes como fichas técnicas, autoria, licenças e seções institucionais.
- Salvar os parágrafos limpos em formatos compatíveis com análise tabular (Excel e CSV).

### Critérios de validade de parágrafos

Um parágrafo é considerado **inválido** se:
- Contém menos de 5 palavras;
- É composto apenas por números ou bullets (ex: "•", "*");
- Contém **dois ou mais termos institucionais** como "ministério", "licença", "tiragem", "autores", etc.

### Funcionalidades do script

- `quebrar_paragrafos_brutos()`: divide o texto com base em pontuação e letras maiúsculas, simulando a quebra natural de parágrafos.
- `paragrafo_valido()`: aplica os filtros para garantir que apenas conteúdo relevante permaneça.
- `limpar_caracteres_invalidos()`: remove símbolos que podem causar erro na exportação para Excel.

### Arquivos gerados

- **`paragrafos_validos_corpus.xlsx`**: contém os parágrafos válidos organizados por arquivo e número do parágrafo.
- **`paragrafos_validos_corpus.csv`**: versão em CSV dos mesmos dados.
- **`diagnostico_paragrafos_por_arquivo.xlsx`**: relatório com o total de parágrafos lidos, quantos foram mantidos e quantos foram descartados por arquivo.

### Exemplo de estrutura da saída:

| Arquivo               | Número do parágrafo | Parágrafo                            |
|-----------------------|---------------------|--------------------------------------|
| dengue_guia2023.txt   | 1                   | A dengue é uma doença viral...       |
| dengue_guia2023.txt   | 2                   | A transmissão ocorre pela picada... |

| Arquivo               | Total lidos | Válidos mantidos | Descartados |
|-----------------------|-------------|------------------|-------------|
| dengue_guia2023.txt   | 150         | 87               | 63          |

Essa etapa garante que o corpus textual a ser utilizado em análises futuras esteja **limpo, bem segmentado e sem conteúdos irrelevantes**, servindo como base sólida para tarefas como geração automática de perguntas e avaliação de IA.


In [ ]:
# === CONFIGURAÇÃO ===
PASTA_CORPUS = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")
ARQUIVO_SAIDA = PASTA_CORPUS / "paragrafos_validos_corpus.xlsx"
ARQUIVO_DIAGNOSTICO = PASTA_CORPUS / "diagnostico_paragrafos_por_arquivo.xlsx"

# === FUNÇÕES ===
def paragrafo_valido(paragrafo):
    paragrafo = paragrafo.strip()
    if len(paragrafo.split()) < 5:
        return False
    if re.match(r'^(\d+|[•*])$', paragrafo):  # apenas número ou bullet
        return False
    if sum(1 for palavra in ["ministério", "licença", "tiragem", "elaboração", "cep", "autores", "versão", "editoração"] if palavra in paragrafo.lower()) >= 2:
        return False
    return True

def limpar_caracteres_invalidos(texto):
    return ILLEGAL_CHARACTERS_RE.sub("", texto)

def quebrar_paragrafos_brutos(texto):
    # Substitui quebras de linha simples por espaço (evita quebra errada)
    texto = texto.replace("\n", " ")
    # Insere quebra forçada após ponto final seguido de espaço e letra maiúscula
    texto = re.sub(r'\. (?=[A-ZÁÉÍÓÚÇ])', '.\n\n', texto)
    # Divide onde houver \n\n forçado
    return [p.strip() for p in texto.split('\n\n') if p.strip()]

# === PROCESSAMENTO ===
dados = []
diagnostico = []

for caminho_txt in PASTA_CORPUS.rglob("*.txt"):
    try:
        with open(caminho_txt, "r", encoding="utf-8") as f:
            conteudo = f.read()

        # Divide usando segmentação forçada
        paragrafos_raw = quebrar_paragrafos_brutos(conteudo)
        paragrafos_validos = [p for p in paragrafos_raw if paragrafo_valido(p)]

        for i, paragrafo in enumerate(paragrafos_validos, 1):
            dados.append({
                "Arquivo": caminho_txt.name,
                "Parágrafo": limpar_caracteres_invalidos(paragrafo),
                "Número do parágrafo": i
            })

        diagnostico.append({
            "Arquivo": caminho_txt.name,
            "Total lidos": len(paragrafos_raw),
            "Válidos mantidos": len(paragrafos_validos),
            "Descartados": len(paragrafos_raw) - len(paragrafos_validos)
        })

    except Exception as e:
        print(f"⚠️ Erro ao processar {caminho_txt.name}: {e}")

# === EXPORTAÇÃO ===
df_paragrafos = pd.DataFrame(dados)
df_paragrafos.to_excel(ARQUIVO_SAIDA, index=False)

df_diag = pd.DataFrame(diagnostico)
df_diag.to_excel(ARQUIVO_DIAGNOSTICO, index=False)

ARQUIVO_CSV = PASTA_CORPUS / "paragrafos_validos_corpus.csv"
df_paragrafos.to_csv(ARQUIVO_CSV, index=False, encoding="utf-8")
print(f"📄 Parágrafos válidos salvos em {ARQUIVO_SAIDA}")
print(f"✅ {len(df_paragrafos)} parágrafos válidos salvos em {ARQUIVO_SAIDA}")
print(f"🩺 Diagnóstico salvo em {ARQUIVO_DIAGNOSTICO}")

📄 Parágrafos válidos salvos em C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_validos_corpus.xlsx
✅ 20106 parágrafos válidos salvos em C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_validos_corpus.xlsx
🩺 Diagnóstico salvo em C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\diagnostico_paragrafos_por_arquivo.xlsx


# Consolidação e Resumo dos Arquivos Únicos (.CSV)

<p align="center">
    <img src="https://img.icons8.com/ios-filled/100/document--v1.png" width="80" alt="Arquivos Únicos"/>
</p>

Esta etapa realiza a consolidação e análise dos arquivos presentes em `corpus_final`, utilizando o arquivo de entrada `resumo_corpus_final.csv`, que contém metadados como tamanho, número de linhas e caracteres por arquivo.

### Objetivos da etapa

- **Identificar e manter apenas arquivos únicos**, mesmo quando há nomes duplicados ou corrompidos (ex: `dengue_txttxt.txt`).
- **Agrupar e sumarizar por subpasta**, fornecendo uma visão geral da distribuição de documentos por tema ou doença.

### O que o script faz

- **Importa o CSV de entrada** com os metadados de todos os arquivos `.txt`.
- **Normaliza os nomes de arquivo**, criando a coluna `Arquivo_base` para eliminar duplicações.
- **Filtra os arquivos únicos** por subpasta e nome base.
- **Agrupa por subpasta**, calculando:
  - Total de arquivos únicos
  - Tamanho total (bytes)
  - Média de linhas por arquivo
  - Média de caracteres por arquivo

### Arquivos gerados

- `resumo_arquivos_unicos.csv`: lista dos arquivos únicos por subpasta.
- `resumo_por_subpasta.csv`: tabela de resumo consolidado por subpasta.

| Subpasta    | Total de arquivos únicos | Tamanho total (bytes) | Média de linhas | Média de caracteres |
|-------------|---------------------------|------------------------|------------------|----------------------|
| dengue      | 8                         | 153000                 | 117              | 5300                 |
| tuberculose | 6                         | 94000                  | 98               | 4100                 |

### Observações

- A deduplicação considera tanto o nome quanto a subpasta.
- Os arquivos finais estão em formato `.csv`, permitindo fácil importação por sistemas externos, planilhas e painéis.
- Esta etapa depende da consistência dos nomes gerados nas etapas anteriores (correção e diagnóstico).

O resultado é um panorama claro e confiável da composição do corpus final, essencial para validações e planejamentos posteriores.


In [ ]:
# === CONFIGURAÇÃO ===
PASTA_BASE = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final") #identifica a pasta base onde os arquivos estão localizados
ARQUIVO_ENTRADA = PASTA_BASE / "resumo_corpus_final.csv"# caminho do arquivo CSV de entrada com o resumo dos arquivos
ARQUIVO_UNICOS = PASTA_BASE / "resumo_arquivos_unicos.csv"# caminho do arquivo de saída com os arquivos únicos
ARQUIVO_AGREGADO = PASTA_BASE / "resumo_por_subpasta.csv" # caminho do arquivo de saída com o resumo por subpasta

# === LEITURA DO ARQUIVO DE RESUMO ===
df = pd.read_csv(ARQUIVO_ENTRADA)

# === REMOVER DUPLICATAS POR NOME BASE DO ARQUIVO ===
# Extrai o nome base (sem partes repetidas)
df["Arquivo_base"] = df["Arquivo"].str.extract(r"^(.+?)(?:_?txt)+(?:\.txt)?$", expand=False).str.strip()#aqui, usamos uma expressão regular para extrair o nome base do arquivo, removendo partes repetidas como "_txt" ou ".txt". Isso ajuda a identificar arquivos com nomes semelhantes.
df_unicos = df.drop_duplicates(subset=["Subpasta", "Arquivo_base"])#já aqui, removemos duplicatas com base na subpasta e no nome base do arquivo. Isso garante que apenas um arquivo por subpasta e nome base seja mantido.

# === AGRUPAMENTO POR SUBPASTA ===
# Agrupa os dados por subpasta e calcula as estatísticas desejadas
df_grupo = df_unicos.groupby("Subpasta").agg({
    "Arquivo": "count",
    "Tamanho (bytes)": "sum",
    "Nº de linhas": "mean",
    "Nº de caracteres": "mean"
}).reset_index()

df_grupo = df_grupo.rename(columns={
    "Arquivo": "Total de arquivos únicos",
    "Tamanho (bytes)": "Tamanho total (bytes)",
    "Nº de linhas": "Média de linhas",
    "Nº de caracteres": "Média de caracteres"
})

# === EXPORTAÇÃO ===
# (Removido: df_unicos.to_excel(ARQUIVO_UNICOS, index=False))
df_unicos.to_csv(ARQUIVO_UNICOS, index=False, encoding="utf-8")
df_grupo.to_csv(ARQUIVO_AGREGADO, index=False, encoding="utf-8")

print(f"✅ Arquivo único salvo em: {ARQUIVO_UNICOS}")
print(f"📊 Resumo por subpasta salvo em: {ARQUIVO_AGREGADO}")

ERROR! Session/line number was not unique in database. History logging moved to new session 39
✅ Arquivo único salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\resumo_arquivos_unicos.xlsx
📊 Resumo por subpasta salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\resumo_por_subpasta.xlsx


# Geração do Corpus Unificado (`corpus_unificado.txt`)

<p align="center">
  <img src="https://img.icons8.com/ios-filled/100/idea.png" width="80" alt="Lâmpada - Ideia"/>
</p>

Após as etapas de extração, limpeza, deduplicação e validação dos arquivos `.txt`, esta etapa consolida todo o conteúdo textual da pasta `corpus_final` em um único arquivo chamado `corpus_unificado.txt`.

### Objetivos da etapa

- **Agrupar todos os arquivos limpos** em um único arquivo de texto.
- **Preservar a estrutura organizacional** dos documentos, mantendo a identificação por subpasta (tema/doença) e nome do arquivo.
- **Facilitar a inspeção visual**, revisão manual ou busca textual do corpus consolidado.

### O que o script faz

- Percorre recursivamente todos os arquivos `.txt` da pasta `corpus_final`.
- Ignora arquivos muito curtos (menos de 50 caracteres), que provavelmente são inválidos ou vazios.
- Para cada arquivo válido:
  - Lê o conteúdo e remove espaços em branco no início e fim.
  - Identifica o nome do arquivo e a subpasta a que pertence.
  - Adiciona ao `corpus_unificado.txt` com um cabeçalho no formato:

    ### Arquivo: nome_do_arquivo.txt | Subpasta: nome_da_subpasta
    ```

- Salva o resultado em `corpus_final/corpus_unificado.txt`.

### Finalidade do arquivo `corpus_unificado.txt`

- Permite uma **validação rápida do conteúdo final** do corpus.
- Pode ser aberto em qualquer editor de texto para:
  - Revisão por especialistas;
  - Busca de termos relevantes;
  - Conferência de organização e integridade textual;
  - Geração de índices ou relatórios.

### Exemplo de estrutura no arquivo final

Arquivo: dengue_transmissao.txt | Subpasta: dengue
A dengue é uma doença viral transmitida principalmente por mosquitos do gênero Aedes...

Arquivo: hanseniase_tratamento.txt | Subpasta: hanseniase
O tratamento da hanseníase é ofertado gratuitamente pelo SUS e baseado na poliquimioterapia...


### Observações

- O script pode ser executado sempre que houver atualização nos textos limpos.
- Ele não altera os arquivos originais, apenas lê e concatena.
- Útil para revisar o corpus como um todo sem precisar abrir os arquivos individualmente.

Esta etapa finaliza a construção do corpus textual unificado, sendo um ponto de partida para processos de análise automática, revisão qualitativa ou auditorias de conteúdo.

In [ ]:
# === CONFIGURAÇÕES ===
PASTA_CORPUS = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")# identifica a pasta base onde os arquivos estão localizados
ARQUIVO_SAIDA = PASTA_CORPUS / "corpus_unificado.txt"# caminho do arquivo de saída onde o corpus unificado será salvo

# === UNIFICAR CONTEÚDO ===
with open(ARQUIVO_SAIDA, "w", encoding="utf-8") as arquivo_saida: # abre o arquivo de saída para escrita
    for caminho_txt in PASTA_CORPUS.rglob("*.txt"):
        try:
            with open(caminho_txt, "r", encoding="utf-8") as f:
                conteudo = f.read().strip()

            if len(conteudo) < 50:
                continue  # ignora arquivos muito curtos

            nome_arquivo = caminho_txt.name
            subpasta = caminho_txt.parent.name

            bloco = f"\n\n### Arquivo: {nome_arquivo} | Subpasta: {subpasta}\n\n{conteudo}\n\n"
            arquivo_saida.write(bloco)

        except Exception as e:
            print(f"Erro ao ler {caminho_txt.name}: {e}") # ignora erros de leitura

print(f"✅ Corpus unificado salvo em: {ARQUIVO_SAIDA}")

✅ Corpus unificado salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\corpus_unificado.txt


# <img src="https://img.icons8.com/ios-filled/100/document--v1.png" width="40" style="vertical-align:middle; margin-right:8px;"/> Filtragem Semântica Rígida dos Parágrafos


Esta etapa aplica uma filtragem semântica rigorosa ao conjunto de parágrafos válidos previamente extraídos, com o objetivo de manter **apenas os trechos realmente relevantes para análise textual ou treinamento de modelos de IA**.

### Objetivos da etapa

- **Descartar parágrafos institucionais ou técnicos irrelevantes**, como autoria, endereço, ficha catalográfica e licenciamento.
- **Evitar parágrafos com conteúdo residual**, como listas de anexo, sumário ou campos vazios.
- **Aplicar um filtro mais protetivo para segurança e qualidade** do corpus final.

### Funcionalidades do script

- **Leitura do arquivo `paragrafos_validos_corpus.csv`**, que contém todos os parágrafos classificados como válidos em etapas anteriores.
- **Aplicação de filtros baseados em regras linguísticas e estruturais**, incluindo:
  - Tamanho máximo de 3000 caracteres.
  - Exclusão por presença de padrões indesejáveis (ex: “nome do autor”, “www.”, “ministério da saúde”, “tiragem”, “índice”, etc.).
  - Exclusão de parágrafos com menos de 6 palavras.

- **Separação dos dados** em dois conjuntos:
  - `paragrafos_filtrados_corpus.csv`: apenas os parágrafos considerados relevantes.
  - `paragrafos_descartados.csv`: parágrafos eliminados por não atenderem aos critérios.

- **Geração de um relatório quantitativo (`relatorio_filtragem_paragrafos.csv`)** com estatísticas resumidas do processo de filtragem.

### Exemplo de padrões que são filtrados

| Tipo de conteúdo excluído           | Exemplo de padrão identificado                     |
|-------------------------------------|----------------------------------------------------|
| Identificação institucional         | `Ministério da Saúde`, `Coordenação-Geral`        |
| Autoria ou ficha técnica            | `Autores:`, `Revisão:`, `Nome do autor:`          |
| Informações logísticas e endereço  | `CEP:`, `Quadra`, `www.saude.gov.br`              |
| Referências editoriais             | `Esta obra é licenciada...`, `Edição 2023`        |
| Tópicos não informativos           | `Anexo`, `Índice`, `Apêndice`                     |

### Arquivos gerados

- **`paragrafos_filtrados_corpus.csv`** – Corpus final com apenas parágrafos relevantes.
- **`paragrafos_descartados.csv`** – Arquivo com todos os parágrafos excluídos na filtragem.
- **`relatorio_filtragem_paragrafos.csv`** – Relatório estatístico com o total de parágrafos, mantidos, excluídos e suas respectivas proporções.

| Métrica         | Valor     |
|-----------------|-----------|
| Total original  | 12.000    |
| Total mantido   | 7.400     |
| Total excluído  | 4.600     |
| % mantido       | 61.67%    |
| % excluído      | 38.33%    |

### Observações

- O filtro é propositalmente conservador, priorizando precisão em detrimento de abrangência.
- Os critérios podem ser ajustados posteriormente conforme o feedback da equipe técnica ou dos especialistas em conteúdo.

Esta etapa garante que o corpus final esteja limpo, consistente e focado no conteúdo técnico-informativo, evitando ruídos que comprometam a qualidade da análise ou da aplicação de inteligência artificial.


In [ ]:
# === CAMINHOS ===
PASTA = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final" # identifica a pasta base onde os arquivos estão localizados
ARQUIVO_ENTRADA = os.path.join(PASTA, "paragrafos_validos_corpus.csv") # caminho do arquivo CSV de entrada com os parágrafos válidos
ARQUIVO_SAIDA = os.path.join(PASTA, "paragrafos_filtrados_corpus.csv") # caminho do arquivo CSV de saída com os parágrafos filtrados
ARQUIVO_RELATORIO = os.path.join(PASTA, "relatorio_filtragem_paragrafos.csv") # caminho do arquivo de relatório com estatísticas da filtragem

# === FILTRO SEMÂNTICO COM PROTEÇÕES RÍGIDAS ===
def paragrafo_relevante(texto):
    texto_limpo = str(texto).strip()

    if len(texto_limpo) > 3000:  # ainda mais seguro
        return False
    if ILLEGAL_CHARACTERS_RE.search(texto_limpo):
        return False

    texto_limpo = texto_limpo.lower()
    padroes_excluir = [
        r'nomes?[:\-]',
        r'e-mail|site:|www\.',
        r'cep:|quadra|srtv|endere',
        r'coordenação-geral|departamento',
        r'ministério da saúde',
        r'esta obra.*licença',
        r'(nome|cargo) do autor',
        r'saúde pública brasileira.*acesso',
        r'\b(anexo|apêndice|índice)\b',
        r'(autores?|versão|tiragem|edição).*202[0-9]',
        r'(sigla|significado) de ',
        r'(organizador|responsável|revisão)',
    ]
    for padrao in padroes_excluir:
        if re.search(padrao, texto_limpo):
            return False
    if len(texto_limpo.split()) < 6:
        return False
    return True

# === LEITURA ===
df = pd.read_csv(ARQUIVO_ENTRADA)
df["Relevante"] = df["Parágrafo"].astype(str).apply(paragrafo_relevante)

# === SEPARAÇÃO ===
df_filtrado = df[df["Relevante"]].drop(columns=["Relevante"])
df_excluido = df[~df["Relevante"]].drop(columns=["Relevante"])

# === SALVAR RESULTADOS EM .CSV ===
df_filtrado.to_csv(ARQUIVO_SAIDA, index=False, encoding="utf-8")
df_excluido.to_csv(PASTA + "/paragrafos_descartados.csv", index=False, encoding="utf-8")

# === RELATÓRIO RESUMIDO ===
resumo = pd.DataFrame([{
    "Total original": len(df),
    "Total mantido": len(df_filtrado),
    "Total excluído": len(df_excluido),
    "% mantido": round(100 * len(df_filtrado) / len(df), 2),
    "% excluído": round(100 * len(df_excluido) / len(df), 2),
}])
resumo.to_csv(ARQUIVO_RELATORIO, index=False)

print(f"✅ CSV salvo com {len(df_filtrado)} parágrafos: {ARQUIVO_SAIDA}")
print(f"📊 Relatório salvo: {ARQUIVO_RELATORIO}")



✅ CSV salvo com 17640 parágrafos: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_filtrados_corpus.csv
📊 Relatório salvo: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\relatorio_filtragem_paragrafos.csv


# <img src="https://img.icons8.com/ios-filled/100/ask-question.png" width="40" style="vertical-align:middle; margin-right:8px;"/> Geração de Perguntas com Modelo T5

Esta etapa aplica **geração automática de perguntas** a partir dos parágrafos previamente filtrados, utilizando o modelo de linguagem `valhalla/t5-base-qg-hl`. O objetivo é criar uma base de questões que poderá ser usada para treinamento, avaliação de IA ou análise de compreensão textual.

O **T5 (Text-to-Text Transfer Transformer)** é um modelo de linguagem desenvolvido pelo Google que converte todas as tarefas de processamento de linguagem natural em um formato de entrada e saída de texto para texto. Isso significa que, para tarefas como tradução, resumo, resposta a perguntas ou geração de perguntas, o T5 recebe uma instrução textual e retorna uma resposta também em texto. Sua arquitetura baseada em transformers permite grande flexibilidade e desempenho em múltiplas tarefas, sendo amplamente utilizado para aplicações avançadas de NLP.

### Objetivos da etapa

- Gerar uma pergunta para cada parágrafo considerado relevante no corpus.
- Dividir o processo em blocos para **evitar sobrecarga de memória e facilitar retomadas**.
- Salvar os resultados em arquivos separados por lote.

### Funcionalidades do script

- **Carregamento do modelo T5** e tokenizer (`valhalla/t5-base-qg-hl`) com suporte a GPU (caso disponível).
- **Leitura do arquivo `paragrafos_filtrados_corpus.csv`**, contendo os textos-base para geração.
- **Divisão do corpus em blocos de 1000 parágrafos**, controlando o tamanho dos lotes para evitar falhas em máquinas com menos memória.
- **Geração de perguntas** com base na instrução `generate question: <texto>`, em que o modelo entende que deve formular uma pergunta sobre o conteúdo apresentado.
- **Exportação dos blocos** em arquivos CSV nomeados sequencialmente, por exemplo: `perguntas_001.csv`, `perguntas_002.csv`, etc.
- **Verificação automática de blocos já processados**, evitando retrabalho em execuções futuras.

### Estrutura do arquivo gerado por bloco

| Parágrafo                                                     | Pergunta                                      |
|---------------------------------------------------------------|-----------------------------------------------|
| A dengue é uma doença viral transmitida por mosquitos...      | Qual é o modo de transmissão da dengue?       |
| O tratamento da tuberculose dura no mínimo seis meses...      | Quanto tempo dura o tratamento da tuberculose?|

### Organização dos resultados

- As perguntas são salvas em uma subpasta com data do dia de execução, ex:  
  `corpus_final/perguntas_geradas_20250803/`
- Cada arquivo contém até 1000 pares de parágrafo/pergunta.
- A pasta de saída é criada automaticamente, se ainda não existir.

### Observações

- A geração é feita com **máximo de 64 tokens** por pergunta para garantir objetividade.
- O script pode ser executado novamente sem sobrescrever resultados já salvos, tornando-o seguro para execuções parciais.
- O modelo `valhalla/t5-base-qg-hl` foi escolhido por seu bom desempenho em tarefas de question generation em português e inglês, mas pode ser substituído se necessário.

Essa etapa transforma o corpus textual em uma base estruturada de perguntas, pronta para ser usada em avaliações manuais, benchmarking de IA, ou sistemas de apoio ao ensino e treinamento.


In [ ]:
%pip install --upgrade torch --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import time
# === CONFIGURAÇÕES ===
PASTA = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final" # Identifica a pasta base onde os arquivos estão localizados
ARQUIVO_ENTRADA = os.path.join(PASTA, "paragrafos_filtrados_corpus.csv") # Caminho do arquivo CSV de entrada com os parágrafos filtrados
DATA = datetime.today().strftime('%Y%m%d') # Obtém a data atual no formato YYYYMMDD
PASTA_SAIDA = os.path.join(PASTA, f"perguntas_geradas_{DATA}")# Caminho da pasta de saída onde as perguntas geradas serão salvas
os.makedirs(PASTA_SAIDA, exist_ok=True)# Cria a pasta de saída se não existir

BLOCO_TAMANHO = 1000 #define o tamanho do bloco de processamento, ou seja, quantas linhas serão processadas por vez

# === MODELO DE PERGUNTAS ===
modelo = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer = T5Tokenizer.from_pretrained(modelo) 
model = T5ForConditionalGeneration.from_pretrained(modelo) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) 

# === FUNÇÃO DE GERAÇÃO ===
def gerar_pergunta(texto):
    entrada = f"generate question: {texto}"
    inputs = tokenizer(entrada, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(inputs["input_ids"], max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# === CARREGAR DADOS ===
df = pd.read_csv(ARQUIVO_ENTRADA).reset_index(drop=True)
total_linhas = len(df)
num_blocos = (total_linhas + BLOCO_TAMANHO - 1) // BLOCO_TAMANHO  # Arredonda para cima

# === PROCESSAMENTO POR BLOCO ===
# === PROCESSAMENTO POR BLOCO ===
for i in range(num_blocos):
    inicio_tempo = time.time()
    inicio = i * BLOCO_TAMANHO
    fim = min((i + 1) * BLOCO_TAMANHO, total_linhas)
    bloco_df = df.iloc[inicio:fim].copy()
    saida_parcial = os.path.join(PASTA_SAIDA, f"perguntas_{i+1:03}.csv")

    # Libera memória da GPU ANTES
    torch.cuda.empty_cache()

    # Pular se já existir
    if os.path.exists(saida_parcial):
        print(f"⏭️ Bloco {i+1} já processado: {saida_parcial}")
        continue

    print(f"🔄 Gerando perguntas para bloco {i+1} ({inicio}–{fim})...")

    perguntas = []
    for paragrafo in tqdm(bloco_df["Parágrafo"].astype(str), desc=f"Bloco {i+1}"):
        try:
            pergunta = gerar_pergunta(paragrafo)
        except Exception as e:
            pergunta = f"[Erro: {str(e)}]"
        perguntas.append(pergunta)

        # Libera GPU entre cada geração, se necessário
        torch.cuda.empty_cache()

    fim_tempo = time.time()
    print(f"⏱️ Tempo gasto no bloco {i+1}: {fim_tempo - inicio_tempo:.2f} segundos")

    bloco_df["Pergunta"] = perguntas
    bloco_df.to_csv(saida_parcial, index=False, encoding="utf-8")
    print(f"✅ Perguntas salvas em: {saida_parcial}")

⏭️ Bloco 1 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_001.csv
⏭️ Bloco 2 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_002.csv
⏭️ Bloco 3 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_003.csv
⏭️ Bloco 4 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_004.csv
⏭️ Bloco 5 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_005.csv
⏭️ Bloco 6 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_006.csv
⏭️ Bloco 7 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_007.csv
⏭️ Bloco 8 já processado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250804\perguntas_008.csv
⏭️ Bloco 9 já processado

Bloco 11:   4%|▎         | 36/1000 [00:13<04:33,  3.52it/s]